In [1]:
using PyCall,QuantumOptics,LinearAlgebra,SparseArrays

In [2]:
py"""

import matplotlib.pyplot as plt
import numpy as np
sqrt = np.sqrt
a1_vec = np.array([2,0]);a2_vec = np.array([1,sqrt(3)])
l1 = 3;l2 = 2;L = l1*l2;N = 3*L 
b1=np.array([0,0]);b2=a1_vec/2;b3=a2_vec/2
Basis = [b1,b2,b3]
basis_colors=['red','blue','green']
coordinates = [];sites = [];atom_dict={}
for i1 in range(l1):
    for i2 in range(l2):
        Lp = a1_vec * i1 + a2_vec * i2
        center = a1_vec * i1 + a2_vec * i2 + b1
        shift = (-b2-b3)/sqrt(3)/2
        P1=np.array([center+shift,center+a1_vec+shift,center+a2_vec+shift])
        for ib, b in enumerate(Basis):
            atom_vec = Lp + b
            atom_no = 3 * i1 * l2 + 3 * i2 + ib
            site = [i1,i2,ib]
            sites.append(site)
            coordinates.append(atom_vec)
            atom_dict[tuple(site)]=atom_vec
NN = [[(0,0,1), (0,0,2), (-1,0,1),(0,-1,2),   (-1,0,2), (-1,1,1), (0,-1,1),(1,-1,2)], 
      [(0,0,-1),(0,0,1), (1,0,-1),(1,-1,1),   (0,-1,1), (1,-1,-1),(0,1,-1),(1,0,1)], 
      [(0,0,-1),(0,0,-2),(0,1,-2),(-1,1,-1),  (-1,1,-2),(-1,0,-1),(0,1,-1),(1,0,-2)]
      ]
# t1=L1=-1;t2=L2=0
# t1=L1=0;t2=L2=-1
t1 = -1;L1 = 0.28;t2 = 0.3;L2 = 0.2
# t1=t2=-1;L1=L2=0
hopps = [[t1+1j*L1,t1-1j*L1,t1+1j*L1,t1-1j*L1,  t2+1j*L2,t2-1j*L2,t2-1j*L2,t2+1j*L2], 
         [t1-1j*L1,t1+1j*L1,t1-1j*L1,t1+1j*L1,  t2-1j*L2,t2+1j*L2,t2+1j*L2,t2-1j*L2], 
         [t1-1j*L1,t1+1j*L1,t1+1j*L1,t1-1j*L1,  t2-1j*L2,t2+1j*L2,t2+1j*L2,t2-1j*L2]] 
H = np.zeros([N,N],dtype=complex)
for atom_no in range(N):
    atom_site=sites[atom_no]
    for i_delta, delta in enumerate(NN[atom_site[2]]):
        neighbor_site = np.array(atom_site)+np.array(delta)
        neighbor_site[0] = neighbor_site[0]%l1
        neighbor_site[1] = neighbor_site[1]%l2    
        neighbor_no=3*neighbor_site[0]*l2+3*neighbor_site[1]+neighbor_site[2]
        H[neighbor_no,atom_no]=hopps[atom_site[2]][i_delta] 

"""

In [3]:
Nx = py"l1";Ny = py"l2";N = py"N";t1 = py"t1";L1 = py"L1";t2 = py"t2";L2 = py"L2"
println("Parameters:")
println("Size of Lattice=",Nx,"X",Ny)
println("Number of Sites=",N)
println("Hopping Amplitudes: t1=",t1,",L1=",L1,",t2=",t2,",L2=",L2)
H = PyArray(py"H"o)
eigvals(H)

Parameters:
Size of Lattice=3X2
Number of Sites=18
Hopping Amplitudes: t1=-1,L1=0.28,t2=0.3,L2=0.2


18-element Vector{Float64}:
 -2.7999999999999994
 -2.7715699522112027
 -2.7683070420017826
 -2.7683070420017812
 -2.757774380143503
 -2.7577743801434993
 -0.14029952547266727
 -0.14029952547266653
  5.39137334129462e-16
  0.3035219724976965
  0.3035219724976971
  1.1228718707889795
  1.6771281292110203
  2.454252407645805
  2.4542524076458054
  2.7715699522111983
  2.9086065674744477
  2.9086065674744486

In [26]:
NBasis = NLevelBasis(N)

NLevel(N=18)

In [27]:
# SINGLE PARTICLE
Hamiltonian_sp = SparseOperator(NBasis)
for m in 1:N
    for n in 1:N
        Hamiltonian_sp = Hamiltonian_sp + H[m,n] * transition(NBasis, m, n)
    end
end
eigenenergies(dense(Hamiltonian_sp))

18-element Vector{Float64}:
 -2.7999999999999994
 -2.7715699522112027
 -2.7683070420017826
 -2.7683070420017812
 -2.757774380143503
 -2.7577743801434993
 -0.14029952547266727
 -0.14029952547266653
  5.39137334129462e-16
  0.3035219724976965
  0.3035219724976971
  1.1228718707889795
  1.6771281292110203
  2.454252407645805
  2.4542524076458054
  2.7715699522111983
  2.9086065674744477
  2.9086065674744486

eigvals(H) == EigenValues (TRUE!)

In [28]:
# MANY-BODY #1
PN=3
States_mb = fermionstates(NBasis, [PN]) 
Basis_mb = ManyBodyBasis(NBasis, States_mb)
Hamiltonian_mb = SparseOperator(Basis_mb) 
for m in 1:N
    for n in 1:N
        Hamiltonian_mb = Hamiltonian_mb + H[m,n] * transition(Basis_mb, m, n)
    end
end
eigenenergies(dense(Hamiltonian_mb))    

816-element Vector{Float64}:
 -8.3335461745941
 -8.291149615862603
 -7.877884327968178
 -7.8668045728385545
 -7.8668045728385465
 -7.829008803294748
 -7.829008803294729
 -7.792323219199143
 -7.772350158439619
 -7.772350158439603
 -7.7347264299447485
 -7.7347264299447405
 -7.681000002245724
  ⋮
  7.25444851520318
  7.417740150599752
  7.428348536102649
  7.42913928936015
  7.4524893812754245
  7.452489381275436
  7.466343698512413
  7.466343698512444
  7.659056683403624
  7.659056683403707
  7.757586114015492
  8.19791978214602

In [25]:
# MANY-BODY #2
H_mb = manybodyoperator(Basis_mb, Hamiltonian_sp)
eigenenergies(dense(H_mb))    

816-element Vector{Float64}:
 -8.3335461745941
 -8.291149615862603
 -7.877884327968178
 -7.8668045728385545
 -7.8668045728385465
 -7.829008803294748
 -7.829008803294729
 -7.792323219199143
 -7.772350158439619
 -7.772350158439603
 -7.7347264299447485
 -7.7347264299447405
 -7.681000002245724
  ⋮
  7.25444851520318
  7.417740150599752
  7.428348536102649
  7.42913928936015
  7.4524893812754245
  7.452489381275436
  7.466343698512413
  7.466343698512444
  7.659056683403624
  7.659056683403707
  7.757586114015492
  8.19791978214602

CONCLUSION: #1 Metod faster than #2 method